In [ ]:
# Step 1: Install necessary libraries
!pip install requests pandas

# Step 2: Make an API request to obtain a list of weather stations and their coordinates
import requests

url = 'https://geo.weather.gc.ca/geomet/?lang=E&service=WFS&request=GetFeature&version=2.0.0&typeName=msc:weather_station'
response = requests.get(url)

# Parse the XML response to extract station coordinates
from xml.etree import ElementTree

tree = ElementTree.fromstring(response.content)
namespaces = {'wfs': 'http://www.opengis.net/wfs/2.0'}
stations = {}

for station in tree.findall('.//wfs:member/msc:weather_station', namespaces):
    code = station.find('msc:station_name', namespaces).text.strip()
    lat = float(station.find('msc:latitude', namespaces).text)
    lon = float(station.find('msc:longitude', namespaces).text)
    stations[code] = (lat, lon)

# Step 3: Find the closest weather station to the given address
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

address = '560 Rochester St, Ottawa, ON, CA'
geolocator = Nominatim(user_agent='myapp')
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude

closest_station = ''
closest_distance = float('inf')
for code, (station_lat, station_lon) in stations.items():
    distance = geodesic((lat, lon), (station_lat, station_lon)).km
    if distance < closest_distance:
        closest_station = code
        closest_distance = distance

print(f'Closest station: {closest_station}, distance: {closest_distance:.2f} km')

# Step 4: Make an API request to obtain the daily climate observations for the chosen station for the entirety of 2011
params = {
    'service': 'WFS',
    'version': '2.0.0',
    'request': 'GetFeature',
    'typeName': 'msc:daily_climate_observations',
    'outputFormat': 'json',
    'filter': f"msc:station_name='{closest_station}' and msc:climate_identifier='1' and msc:year='2011'"
}
response = requests.get(url, params=params)

# Step 5: Store the data in a pandas dataframe for further analysis
import json
import pandas as pd

data = json.loads(response.content)
observations = data['features']
df = pd.json_normalize(observations)
df = df.drop(columns=['type', 'id', 'geometry.type', 'geometry.coordinates', 'properties.station_name'])
df.columns = df.columns.str.replace('properties.', '')
df = df.set_index('date')
df.index = pd.to_datetime(df.index)